<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsquedas WOS+SCI+SCP+PTJ+CTR para UdeA

Búsquedas en bases bibligráficas  
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
de los artículos científicos de la UdeA

La base de datos se creó con:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

## functions

In [2]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel/json or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j

Overwriting drive.cfg


##  Load data bases

In [5]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

In [7]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [135]:
#from check_quality import *
#check_quality(UDEA)

## Indices:
Información obtenida de la columna: `json_column='UDEA_authors'`

In [136]:
json_column='UDEA_authors'

Que contiene listas de diccionarios con la información del autor UDEA: 

`{'DEPARTAMENTO': 'Instituto de Biología',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Sin Grupo Asociado',
  'INICIALES': 'I.',
  'NOMBRE COMPLETO': 'Idalyd Fonseca Gonzalez',
  'NOMBRES': 'Idalyd',
  'PRIMER APELLIDO': 'Fonseca',
  'SEGUNDO APELLIDO': 'Gonzalez',
  'WOS_affiliation': ['Univ Antioquia, Colombia.'],
  'WOS_author': ['FONSECA, IDALYD',
   'FONSECA-GONZALEZ, IDALYD',
   'Fonseca-Gonzalez, Idalyd',
   'Fonseca-Gonzalez, I.'],
  'full_name': 'FONSECA GONZALEZ IDALYD'}`

# Resultados totales

Artículos no identificados:

In [141]:
UDEA_NOT=UDEA[UDEA[json_column]==''].reset_index(drop=True)
UDEA_NOT.shape[0]

4019

Artículos identificados

In [146]:
UDEA_YES=UDEA[UDEA[json_column]!=''].reset_index(drop=True)
UDEA_YES.shape[0]

11681

### Análisis sobre artículos identificados

In [145]:
def extract_key(df,key):
    l=df[json_column].apply(lambda l: np.unique([ d.get(key) for d in l 
                                if d.get(key) ]) if type(l)==list else l).apply(
                       pd.Series).stack().values
    return pd.DataFrame( {key:list(l)} ).groupby(key)[key].count().sort_values(ascending=False)

In [134]:
extract_key(UDEA,'FACULTAD')

FACULTAD
                                            4019
Facultad de Medicina                        3078
Facultad de Ciencias Exactas y Naturales    2255
Facultad de Ingeniería                      1778
Facultad de Ciencias Agrarias                656
Facultad de Ciencias Sociales y Humanas      215
Facultad de Artes                             13
Name: FACULTAD, dtype: int64

In [149]:
extract_key(UDEA_YES,'DEPARTAMENTO')

DEPARTAMENTO
Departamento de Microbiología y Parasitología                   885
Instituto de Física                                             862
Instituto de Investigaciones Médicas                            719
Instituto de Biología                                           656
Instituto de Química                                            645
Departamento de Medicina Interna                                642
Departamento de  Producción Agropecuaria                        406
Departamento de Pediatría y Puericultura                        388
Departamento de Ingeniería Metalúrgica                          334
Departamento de Ingeniería Sanitaria  y Ambiental               317
Escuela de Medicina Veterinaria                                 305
Departamento de Ingeniería Mecánica                             279
Departamento de Ingeniería Quimica                              276
Departamento de Cirugía                                         210
Departamento de Fisiología         

In [150]:
extract_key(UDEA_YES,'GRUPO')

GRUPO
Sin Grupo Asociado                                                                                                                                                                                436
Grupo de Materia Condensada-UdeA                                                                                                                                                                  261
Inmunovirología                                                                                                                                                                                   235
Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección                                                                         232
Grupo de Manejo Eficiente de la Energía, GIMEL                                                                                                                                                    219
Grup

In [148]:
extract_key(UDEA_YES,'full_name')

full_name
DUQUE ECHEVERRI CARLOS ALBERTO        261
BEDOYA BERRIO GABRIEL DE JESUS        123
LOPERA RESTREPO FRANCISCO JAVIER      120
CERON MUÑOZ MARIO FERNANDO            120
RUGELES LOPEZ MARIA TERESA            115
JAIMES BARRAGAN FABIAN ALBERTO        113
CARMONA FONSECA JAIME DE JESUS        113
PEÑUELA MESA GUSTAVO ANTONIO          110
VELEZ BERNAL IVAN DARIO               110
OLIVERA ANGEL MARTHA EUFEMIA          101
AMARILES MUÑOZ PEDRO JOSE              95
RIOS LUIS ALBERTO                      92
RESTREPO BETANCUR LUIS FERNANDO        92
CARDONA MAYA WALTER DARIO              91
CARDONA ARIAS JAIBERTH ANTONIO         91
ARDILA MEDINA CARLOS MARTIN            86
MORALES ARAMBURO ALVARO LUIS           84
BLAIR TRUJILLO SILVIA VICTORIA         82
ROBLEDO RESTREPO SARA MARIA            80
MONDRAGON PEREZ FANOR                  79
CADAVID JARAMILLO ANGELA PATRICIA      78
AGUDELO SUAREZ ANDRES ALONSO           77
CORNEJO OCHOA JOSE WILLIAM             75
TRIANA CHAVEZ OMAR      

# Búsquedas

In [10]:
#TODO: Improve indices
def get_groups(l,g):
    for d in l:
        gt=d.get('GRUPO')
        if gt and type( gt )==str:
            gs=gt.replace(
                ', Grupo','; Grupo'
            ).split('; ')
            for gg in gs:
                if gg not in g:
                    g.append(gg)
    return g


facultades={'key':'FACULTAD',
            'values' : UDEA.UDEA_authors.apply(lambda l: 
                         [d.get('FACULTAD') for d in l] 
                          if type(l)==list else None
                          ).dropna().apply(pd.Series).stack().unique()}
departamentos={'key':'DEPARTAMENTO',
            'values' :UDEA.UDEA_authors.apply(lambda l: 
                         [d.get('DEPARTAMENTO') for d in l] 
                          if type(l)==list else None
                        ).dropna().apply(pd.Series).stack().unique()}
g=[]
#append to g
tmp=UDEA.UDEA_authors.apply(lambda l: 
                        get_groups(l,g)
        if type(l)==list else None
                        )
grupos={'key':'GRUPO',
            'values' :g}
nombre_completo={'key'    : 'NOMBRE COMPLETO',
                 'values' : UDEA[json_column].apply(lambda l: 
                            [d.get('NOMBRE COMPLETO') for d in l] 
                            if type(l)==list else None
                            ).dropna().apply(pd.Series).stack().unique()
                }#Nombres Apellidos. #TODO: Be sure to include full list
full_name={'key'    : 'full_name',
                 'values' : UDEA[json_column].apply(lambda l: 
                            [d.get('full_name') for d in l] 
                            if type(l)==list else None
                            ).dropna().apply(pd.Series).stack().unique()
                }#Apellidos Nombres . #TODO: Be sure to include full list

## Función de bísqueda

For value string or list of each dictionary within a list of dictionaries, like the column 'UDEA_authors' in `UDEA` DataFrame

In [11]:
def query_json_column(q,df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,**kwargs):
    #Found best exact match from index
    fchoices=fwp.extractOne(q,choices['values'],scorer=scorer)[0]
    # Exact search in indexed subcolumn converted to strins (e.g list → string if necessary)
    dfF=df[df[json_column].apply(lambda l: True in [ str(d.get(choices['key'])).find(fchoices)>-1 
                                    for d in l if d.get(choices['key'])] if type(l)==list else False)]
    return dfF.reset_index(drop=True)

### Autor

In [19]:
r=query_json_column('Diego Alejandro Restrepo Quintero',df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [20]:
r.shape

(37, 181)

In [24]:
r[['TI','AU','authors_WOS',json_column]].reset_index(drop=True)[5:7]

,TI,AU,authors_WOS,UDEA_authors
5,The Fermi-LAT gamma-ray excess at the Galactic Center in the singlet-doublet fermion dark matter model,"Horiuchi, S\nMacias, O\nRestrepo, D\nRivera, A\nZapata, O\nSilverwood, H\n","[{'WOS_author': 'Restrepo, Diego', 'affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21, Medellin, Colombia.'], 'i': 0}, {'WOS_author': 'Rivera, Andres', 'affiliation': ['Univ Antioquia, Inst...","[{'NOMBRES': 'Diego Alejandro', 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'INICIALES': 'D. A.', 'FACULTAD': 'Facultad de Ciencias Exactas y Naturales', 'PRIMER APELLIDO': 'Restrepo',..."
6,Diboson anomaly: Heavy Higgs resonance and QCD vectorlike exotics,"Sierra, DA\nHerrero-Garcia, J\nRestrepo, D\nVicente, A\n","[{'WOS_author': 'Restrepo, D.', 'affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21 Medellin, Medellin, Colombia.'], 'i': 0}]","[{'NOMBRES': 'Diego Alejandro', 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'INICIALES': 'D. A.', 'FACULTAD': 'Facultad de Ciencias Exactas y Naturales', 'PRIMER APELLIDO': 'Restrepo',..."


## Grupos

Ejemplo

In [27]:
r=query_json_column('Grupo de Fenomenología de Interacciones Fundamentales',df=UDEA,json_column='UDEA_authors',
                        choices=grupos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [28]:
r.shape

(82, 181)

Buscar todos

In [29]:
gdf=pd.DataFrame()
for g in grupos['values']:
    r=query_json_column(g,df=UDEA,json_column='UDEA_authors',choices=grupos,
                        scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)
                        
    gdf=gdf.append( {'Group':g,'articles':r.shape[0]},ignore_index=True )
gdf['articles']=gdf['articles'].astype(int)

In [30]:
gdf.sort_values('articles',ascending=False).reset_index(drop=True)[:10]

,Group,articles
0,Sin Grupo Asociado,436
1,Grupo de Materia Condensada-UdeA,300
2,Grupo de Estado Sólido,272
3,Grupo de Neurociencias de Antioquia,240
4,"Grupo de Neurociencias de Antioquia, SINAPSIS",240
5,"Epidemiología, Genética Molecular (GENMOL)",231
6,Genética Molecular (GENMOL),231
7,"Instrumentación Científica y Microelectrónica,",223
8,Instrumentación Científica y Microelectrónica,223
9,"Grupo de Manejo Eficiente de la Energía, GIMEL",219


## Departamento

In [25]:
r=query_json_column('Instituto de Física',df=UDEA,json_column='UDEA_authors',
                        choices=departamentos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [26]:
r.shape

(862, 181)

## Centro

Ejemplo

In [53]:
cen=query_json_column('Facultad de Ciencias Exactas y Naturales',df=UDEA,json_column='UDEA_authors',
                        choices=facultades,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [54]:
cen.shape

(2255, 181)

Todos

In [114]:
fdf=pd.DataFrame()
for f in facultades['values']:
    r=query_json_column(f,df=UDEA,json_column='UDEA_authors',choices=facultades,
                        scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)
    fdf=fdf.append( {'Facultad':f,'articles':r.shape[0]},ignore_index=True )
fdf['articles']=fdf['articles'].astype(int)

In [120]:
fdf.sort_values('articles',ascending=False)

,Facultad,articles
0,Facultad de Medicina,3078
1,Facultad de Ciencias Exactas y Naturales,2255
2,Facultad de Ingeniería,1778
4,Facultad de Ciencias Agrarias,656
3,Facultad de Ciencias Sociales y Humanas,215
5,Facultad de Artes,13
